In [129]:
import os
import requests
import json
import pandas as pd
import yfinance as yf
import pendulum
import matplotlib.pyplot as plt
from MCForecastTools import MCSimulation

%matplotlib inline

In [103]:
import numpy as np
from pandas_datareader import data as wb
import seaborn as sns
from scipy.stats import norm
import datetime as dt
from mc_simulation import MonteCarloSimulation as mcs

In [92]:
#start_date=dt.datetime(2018,1,1)
#end_date=dt.datetime(2020,1,1)
#print(start)

2018-01-01 00:00:00


In [113]:
#ticker = 'TSLA'
#data = pd.DataFrame(['TSLA'])
#wb.DataReader("TSLA", 'yahoo', start=start_date, end=end_date)["Close"]
#Plot
#data.plt(figsize=(15,6))

In [114]:
#price_history = yf.Ticker('TSLA').history(period='2y', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                   #interval='1mo', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                   #actions=False)
#time_series = list(price_history['Close'])
#dt_list = [pendulum.parse(str(dt)).float_timestamp for dt in list(price_history.index)]

#plt.plot(time_series)
#display(dt_list)
#display(time_series)


In [115]:
#political_portfolio_dict_stock_list = [{'Mark_Warner': ['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI']}, 
 #{'Greg_Gianforte': ['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'RHT']},
 #{'Nancy_Pelosi': ['AAPL', 'V', 'DIS', 'CRM', 'META', 'CMCSA']}]
                 
#Mark Warner: SPDR s&p 500 etf, Akre focus fund, t. rowe price group, coho relative value equity fund, vanguard 

#Greg Gianforte: invesco qqq, SPDr s&p 500, vanguard, FICO, Workiva, red hat inc

#Nancy Pelosi: apple, visa, disney, salesforce, facebook, comcast

In [124]:
warner_weights=[0.24, 0.23, 0.22, 0.22, 0.09]
warner_investment_capital=116000000.00

#Warner range of investments from $80m to $348m. Maximum value of assets assumed in calculations.
#investment capital reduced by 2/3 to allow for easier analysis and accomodate top 1/3 of stock 
#investment portfolio. Net worth assumed as investment capital. AMG GW&K Municipal bond fund removed from 
#top 1/3 of stock assets to simplify calculations.

gianforte_weights=[0.26, 0.22, 0.35, 0.08, 0.05, 0.04]
gianforte_investment_capital = 94500000.00

#weights of stocks making up 1/2 of Gianforte investment portfolio. Different Vanguard funds summed. 
#investment capital reduced by 1/2 to allow for easier analysis and accomodate top
#1/2 of stock investment portfolio. Net worth assumed as investment capital. 

pelosi_weights=[0.40, 0.40, 0.08, 0.08, 0.02, 0.02]
pelosi_investment_capital= 64000000.00
                
#Pelosi range of investments from $54m to $256m. Max value investments/assets assumed in calculations. 
#investment capital reduced by 3/4 to allow for easier analysis and accomodate top
#1/4 of stock investment portfolio. Net worth assumed as investment capital.

In [139]:
warner_tickers= ['SPY', 'AKREX', 'TROW', 'COHOX', 'VTI']
gianforte_tickers=['QQQ', 'SPY', 'VTI', 'FICO', 'WK', 'CRM']
pelosi_tickers=['AAPL', 'V', 'DIS', 'CRM', 'META', 'CMCSA']

#timeframe='1Day'

#start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
#end_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()

In [140]:
warner_stock_info=yf.Tickers(warner_tickers)
warner_df=warner_stock_info.history(period='5y')
warner_df=pd.DataFrame(warner_df)
warner_close_data=warner_df['Close']
warner_close_data=warner_close_data.reindex(columns=warner_tickers)
display(warner_close_data)

[*********************100%***********************]  5 of 5 completed


,SPY,AKREX,TROW,COHOX,VTI
Date,,,,,
2018-01-29 00:00:00-05:00,261.374146,30.276783,100.001556,11.290659,134.025452
2018-01-30 00:00:00-05:00,258.693207,30.154554,95.713966,11.117401,132.629181
2018-01-31 00:00:00-05:00,258.821686,30.320431,94.965324,11.030772,132.675125
2018-02-01 00:00:00-05:00,258.527954,30.451389,95.416229,11.001896,132.619965
2018-02-02 00:00:00-05:00,252.899796,29.814074,93.485085,10.770885,129.799850
...,...,...,...,...,...
2023-01-23 00:00:00-05:00,400.630005,49.730000,118.669998,15.780000,201.279999
2023-01-24 00:00:00-05:00,400.200012,49.599998,117.480003,15.760000,200.949997
2023-01-25 00:00:00-05:00,400.350006,49.619999,116.639999,15.840000,201.050003


In [141]:
warner_returns=warner_close_data.pct_change()
mean_warner_returns=warner_returns.mean()
std_warner_returns=warner_returns.std()


In [142]:
MC_30yr = MCSimulation(
    portfolio_data = warner_close_data,
    weights = warner_weights,
    num_simulation = 500,
    num_trading_days = 252*30
)

# Review the simulation input data
MC_30yr.portfolio_data.head()

IndexError: Too many levels: Index has only 1 level, not 2

In [146]:
gianforte_stock_info=yf.Tickers(gianforte_tickers)
gianforte_df=gianforte_stock_info.history(period='5y')
gianforte_df=pd.DataFrame(gianforte_df)
gianforte_close_data=gianforte_df['Close']
gianforte_close_data=gianforte_close_data.reindex(columns= gianforte_tickers)
display(gianforte_close_data)

[*********************100%***********************]  6 of 6 completed


,QQQ,SPY,VTI,FICO,WK,CRM
Date,,,,,,
2018-01-29 00:00:00-05:00,164.340347,261.374176,134.025482,176.130005,22.200001,113.300003
2018-01-30 00:00:00-05:00,162.987762,258.693237,132.629181,173.520004,22.049999,112.230003
2018-01-31 00:00:00-05:00,163.664062,258.821716,132.675110,172.660004,22.299999,113.910004
2018-02-01 00:00:00-05:00,162.272827,258.527893,132.619965,172.839996,22.250000,112.739998
2018-02-02 00:00:00-05:00,159.036255,252.899765,129.799850,169.380005,22.299999,110.779999
...,...,...,...,...,...,...
2023-01-23 00:00:00-05:00,288.959991,400.630005,201.279999,649.950012,92.570000,155.869995
2023-01-24 00:00:00-05:00,288.369995,400.200012,200.949997,640.219971,90.459999,154.860001
2023-01-25 00:00:00-05:00,287.730011,400.350006,201.050003,642.830017,90.580002,156.169998


In [118]:
gianforte_returns=gianforte_close_data.pct_change()
mean_gianforte_returns=gianforte_returns.mean()
std_gianforte_returns=gianforte_returns.std()

In [147]:
pelosi_stock_info=yf.Tickers(pelosi_tickers)
pelosi_df=pelosi_stock_info.history(period='5y')
pelosi_df=pd.DataFrame(pelosi_df)
pelosi_close_data=pelosi_df['Close']
pelosi_close_data=pelosi_close_data.reindex(columns= pelosi_tickers)
display(pelosi_close_data)

[*********************100%***********************]  6 of 6 completed


,AAPL,V,DIS,CRM,META,CMCSA
Date,,,,,,
2018-01-29 00:00:00-05:00,39.928276,120.807625,108.473206,113.300003,185.979996,37.564678
2018-01-30 00:00:00-05:00,39.692932,119.559319,107.082520,112.230003,187.119995,37.886818
2018-01-31 00:00:00-05:00,39.802284,120.217331,105.682114,113.910004,186.889999,38.056847
2018-02-01 00:00:00-05:00,39.885483,121.659203,107.452065,112.739998,193.089996,37.484154
2018-02-02 00:00:00-05:00,38.154850,117.004578,105.711281,110.779999,190.279999,36.786190
...,...,...,...,...,...,...
2023-01-23 00:00:00-05:00,141.110001,224.179993,105.690002,155.869995,143.270004,40.259998
2023-01-24 00:00:00-05:00,142.529999,223.889999,106.000000,154.860001,143.139999,39.750000
2023-01-25 00:00:00-05:00,141.860001,224.899994,108.120003,156.169998,141.500000,40.099998


In [119]:
pelosi_returns=pelosi_close_data.pct_change()
mean_pelosi_returns=pelosi_returns.mean()
std_pelosi_returns=pelosi_returns.std()

In [ ]:
spy_shares_warner= 110476
akrex_shares_warner= 762286
trow_shares_warner= 220000
cohox_shares_warner= 1772222
vti_shares_warner= 74359

qqq_shares_gianforte= 149271
spy_shares_gianforte= 82500
vti_shares_gianforte= 235577
fico_shares_gianforte= 39171
wk_shares_gianforte= 181730
rht_shares_gianforte= 25116

aapl_shares_pelosi=548180
visa_shares_pelosi=193939
dis_shares_pelosi=46694
crm_shares_pelosi=37372
meta_shares_pelosi=7442
cmcsa_shares_pelosi=35774

#total shares calculated based on 2018 median share prices for each stock in portfolio b/c most recent portfolio data
#from 2018. should 

In [ ]:
stock_investment= pelosi_investment_capital * 0.02
#spy 2018 median share price =252
#akrex 2018 median share price = 35
#trow 2018 median share price = 116
#cohox 2018 median share price = 14.4
#vti 2018 median share price = 140.4

#qqq 2018 med share price = 164.6
#fico 2018 med share price = 193
#wk 2018 med share price = 26
#rht 2018 med share price = 150.5

#aapl 2018 med share price = 46.7
#visa 2018 med share price = 132
#disney 2018 med share price = 109.65
#crm 2018 med share price = 137
#meta 2018 med share price = 172
#cmcsa 2018 med share price = 35.78

In [ ]:
stock_investment/35.78

In [ ]:
#chatgpt help:

#There are several steps to creating a Monte Carlo simulation of a stock portfolio in Python:

#Collect historical data for the stocks in the portfolio. This data should include the daily closing prices for each stock.

#Use the collected data to calculate the daily returns for each stock. The daily return is the percentage change in the stock price from one day to the next.

#Use the daily returns to calculate the mean and standard deviation of the returns for each stock. These values will be used to simulate future returns using the Monte Carlo method.

#Use the mean and standard deviation values to generate a large number of random returns for each stock using the normal distribution.

#Use the randomly generated returns to simulate the future performance of the portfolio. This can be done by multiplying the randomly generated returns by the current portfolio weights, and then summing the results for each stock.

#Repeat steps 4 and 5 many times to generate a large number of possible future outcomes for the portfolio.

#Use the simulated outcomes to calculate various statistics, such as the expected return, risk, and the distribution of returns for the portfolio.



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Collect historical data for the stocks in the portfolio
data = pd.read_csv('stock_data.csv')

# Calculate daily returns
returns = data.pct_change()

# Calculate mean and standard deviation of returns
mean_returns = returns.mean()
std_returns = returns.std()

# Use the Monte Carlo method to simulate future returns
num_simulations = 1000
num_days = 365

simulated_returns = pd.DataFrame()
portfolio_cumulative_returns = pd.DataFrame()

for i in range(num_simulations):
    simulated_returns[i] = np.random.normal(mean_returns, std_returns, num_days)

# Calculate the simulated daily returns of the portfolio
portfolio_returns = simulated_returns.T @ weights

# Calculate the cumulative returns of the portfolio
portfolio_cumulative_returns[i] = (1 + portfolio_returns.fillna(0)).cumprod()

# Plot the simulated cumulative returns of the portfolio
plt.figure(figsize=(10,5))
plt.plot(portfolio_cumulative_returns)
plt.xlabel('Simulation')
plt.ylabel('Portfolio Cumulative Returns')
plt.show()